In [1]:
import numpy as np
import pyopencl as cl
import pyopencl.array
import pyopencl.clrandom
# loopy currently requires on pyopencl 
import loopy as lp
lp.set_caching_enabled(False)
from warnings import filterwarnings, catch_warnings
filterwarnings('error', category=lp.LoopyWarning)
ctx = cl.create_some_context(interactive=False)
queue = cl.CommandQueue(ctx)
# Set up pyopencl.Context & CommandQueue

In [2]:
n = 16*16
x_vec_dev = cl.clrandom.rand(queue, n, dtype=np.float32) # device side
y_vec_dev = cl.clrandom.rand(queue, n, dtype=np.float32)
z_vec_dev = cl.clrandom.rand(queue, n, dtype=np.float32)
a_mat_dev = cl.clrandom.rand(queue, (n, n), dtype=np.float32)
b_mat_dev = cl.clrandom.rand(queue, (n, n), dtype=np.float32)
x_vec_host = np.random.randn(n).astype(np.float32) # host side
y_vec_host = np.random.randn(n).astype(np.float32)

In [3]:
knl = lp.make_kernel(
     "{ [i,j,ii,jj]: 0<=i,j,ii,jj<n }",
     """
     out[j,i] = a[i,j] {id=transpose}
     out[ii,jj] = 2*out[ii,jj]  {dep=transpose}
     """)
knl = lp.prioritize_loops(knl, "i,j,ii,jj")
knl = lp.set_options(knl, "write_cl")
print(knl) # Kernel info, including loop domain, instructions and arguments

---------------------------------------------------------------------------
KERNEL: loopy_kernel
---------------------------------------------------------------------------
ARGUMENTS:
a: GlobalArg, type: <runtime>, shape: (n, n), dim_tags: (N1:stride:n, N0:stride:1)
n: ValueArg, type: <runtime>
out: GlobalArg, type: <runtime>, shape: (n, n), dim_tags: (N1:stride:n, N0:stride:1)
---------------------------------------------------------------------------
DOMAINS:
[n] -> { [i, j, ii, jj] : 0 <= i < n and 0 <= j < n and 0 <= ii < n and 0 <= jj < n }
---------------------------------------------------------------------------
INAME IMPLEMENTATION TAGS:
i: None
ii: None
j: None
jj: None
---------------------------------------------------------------------------
INSTRUCTIONS:
  for j, i
↱     out[j, i] = a[i, j]  {id=transpose}
│ end j, i
│ for ii, jj
└     out[ii, jj] = 2*out[ii, jj]  {id=insn}
  end ii, jj
---------------------------------------------------------------------------


In [4]:
evt, (out,) = knl(queue, a=a_mat_dev)
assert (out.get() == a_mat_dev.get().T*2).all()

#define lid(N) ((int) get_local_id(N))
#define gid(N) ((int) get_group_id(N))

__kernel void __attribute__ ((reqd_work_group_size(1, 1, 1))) loopy_kernel(__global float const *__restrict__ a, int const n, __global float *__restrict__ out)
{
  for (int i = 0; i <= -1 + n; ++i)
    for (int j = 0; j <= -1 + n; ++j)
      out[n * j + i] = a[n * i + j];
  for (int ii = 0; ii <= -1 + n; ++ii)
    for (int jj = 0; jj <= -1 + n; ++jj)
      out[n * ii + jj] = 2.0f * out[n * ii + jj];
}



In [5]:
knl = lp.set_options(knl, write_wrapper=True, write_cl=False) # peek at generated code

In [6]:
evt, (out,) = knl(queue, a=x_vec_host)

from __future__ import division

import numpy as _lpy_np
import pyopencl as _lpy_cl
import pyopencl.array as _lpy_cl_array
import pyopencl.tools as _lpy_cl_tools

def _lpy_host_loopy_kernel(_lpy_cl_kernels, queue, a, n, out, wait_for=None, allocator=None):
    from struct import pack as _lpy_pack
    import pyopencl as _lpy_cl
    import pyopencl.tools

    if allocator is None:
        allocator = _lpy_cl_tools.DeferredAllocator(queue.context)


    _global_temporaries = []

    pass
    # {{{ enqueue loopy_kernel

    _lpy_knl = _lpy_cl_kernels.loopy_kernel
    assert _lpy_knl.num_args == 3

    # {{{ process n

    if n is None:
        raise RuntimeError("input argument 'n' must be supplied")
    # cast to Python int to avoid trouble with struct packing or Boost.Python
    n = int(n)

    _lpy_knl.set_arg(1, _lpy_pack('i', n))

    # }}}

    _lpy_knl.set_arg(0, a)
    _lpy_knl.set_arg(2, out)
    _lpy_evt = _lpy_cl.enqueue_nd_range_kernel(queue, _lpy_knl, (1,), (1,),  wait_for=wai

TypeError: shape mismatch on argument 'a' (got: (256,), expected: (256, 256))

In [7]:
lp.show_dependency_graph(knl)

/home/ubuntu/miniconda3/envs/loopydev/lib/python3.6/site-packages/loopy/kernel/tools.py:476: UserWarning: error encountered during scheduling for dep graph -- cannot perform iname clustering: LoopyError(cannot schedule a kernel that has not been preprocessed)
  % (type(e).__name__, e))
